Install Prospect dataset:

In [3]:
%pip install git+https://github.com/wilhelm-lab/PROSPECT

  Cloning https://github.com/wilhelm-lab/PROSPECT to c:\users\micro\appdata\local\temp\pip-req-build-puxbwmb1
  Resolved https://github.com/wilhelm-lab/PROSPECT to commit 46b5e897565d9708f888feed7172904ed6c2d4d1
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Cloning https://github.com/wilhelm-lab/spectrum_fundamentals.git (to revision main) to c:\users\micro\appdata\local\temp\pip-install-ot4co9zx\spectrum-fundamentals_c54e2020da9041c9ad6fd2aefe720d0f
  Resolved https://github.com/wilhelm-lab/spectrum_fundamentals.git to commit a0a8d21bb4196e5a968662664330935b1e512040
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached fastparquet-2023.4.0-cp

  Running command git clone --filter=blob:none --quiet https://github.com/wilhelm-lab/PROSPECT 'C:\Users\micro\AppData\Local\Temp\pip-req-build-puxbwmb1'
  Running command git clone --filter=blob:none --quiet https://github.com/wilhelm-lab/spectrum_fundamentals.git 'C:\Users\micro\AppData\Local\Temp\pip-install-ot4co9zx\spectrum-fundamentals_c54e2020da9041c9ad6fd2aefe720d0f'
  Running command git checkout -b main --track origin/main
  branch 'main' set up to track 'origin/main'.
  Switched to a new branch 'main'

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Download + Import Prospect dataset:

In [4]:
import prospectdataset as prods

In [6]:
#too slow... manually download from
#prods.download_dataset('all', "data/")

In [9]:
import pandas as pd
import os

C:\Users\micro\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\micro\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\micro\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [11]:
df = pd.read_parquet("data/Thermo_SRM_Pool_meta_data.parquet", engine='fastparquet')

In [12]:
print(df.shape)
print(df.head(4))

(6080606, 15)
                                       raw_file  scan_number  \
0  01640c_BC1-Thermo_SRM_Pool_3_01_01-DDA-1h-R2        34736   
1  01640c_BC1-Thermo_SRM_Pool_3_01_01-DDA-1h-R2        25591   
2  01640c_BC1-Thermo_SRM_Pool_3_01_01-DDA-1h-R2        24831   
3  01640c_BC1-Thermo_SRM_Pool_3_01_01-DDA-1h-R2        34870   

      modified_sequence  precursor_charge  precursor_intensity          mz  \
0  LPGSLETYVEQEQGENANDR                 2           29525630.0  1125.01950   
1       HGSLQEYLQNDTGSK                 2           13188580.0   838.89737   
2        VEEEEEINSELTAR                 2           20663460.0   824.38904   
3  LPGSLETYVEQEQGENANDR                 2           19884630.0  1125.01950   

   precursor_mz fragmentation mass_analyzer  retention_time  \
0   1125.018331           CID          ITMS          38.411   
1    838.894401           HCD          FTMS          30.394   
2    824.388401           CID          ITMS          29.737   
3   1125.018831       

In [13]:
print(df.columns)

Index(['raw_file', 'scan_number', 'modified_sequence', 'precursor_charge',
       'precursor_intensity', 'mz', 'precursor_mz', 'fragmentation',
       'mass_analyzer', 'retention_time', 'indexed_retention_time',
       'andromeda_score', 'peptide_length', 'orig_collision_energy',
       'aligned_collision_energy'],
      dtype='object')


Needed: modified_sequence, precursor charge, precursor_intensity (abundance), mz

In [21]:
for index, seq in enumerate(df["modified_sequence"]):
    if seq == "LPGSLETYVEQEQGENANDR":
        print(index, df["precursor_charge"][index], df["scan_number"][index])
        # print df row
        print("---"+ df.iloc[index].to_string())
        print("      ------------------------")

0 2 34736
---raw_file                    01640c_BC1-Thermo_SRM_Pool_3_01_01-DDA-1h-R2
scan_number                                                        34736
modified_sequence                                   LPGSLETYVEQEQGENANDR
precursor_charge                                                       2
precursor_intensity                                           29525630.0
mz                                                             1125.0195
precursor_mz                                                 1125.018331
fragmentation                                                        CID
mass_analyzer                                                       ITMS
retention_time                                                    38.411
indexed_retention_time                                         62.827337
andromeda_score                                                   447.14
peptide_length                                                        20
orig_collision_energy                 